# Méthode de Newton

In [ ]:
using Plots
plotly()

## Newton-Raphson en optimisation

Itération de Newton-Raphson:
$$
x_{k+1} = x_k-\frac{f'(x_k)}{f''(x_k)}
$$

In [ ]:
function Newton(f::Function, df::Function, d2f:: Function,
        xstart::Float64, verbose::Bool = false, store=false,
        δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = xstart
    if (store)
        iter = [ f(x) x ]
    end
    if (verbose)
        fx = f(x)
        println("$k. x = $x, f(x) = $fx")
    end
    dfx = df(x)
    while (abs(dfx) > δ && k < nmax)
        k += 1
        dfx = df(x)
        x = x-dfx/d2f(x)
        if (store)
            iter = [iter ; f(x) x]
        end
        if (verbose)
            fx = f(x)
            println("$k. x = $x, f(x) = $fx")
        end
    end
    
    if (store)
        return iter
    end
end

In [ ]:
func(x) = -10x^2 + 4sin(x) + x^4
dfunc(x) = -20x + 4cos(x) + 4x^3
d2func(x) = -20 - 4sin(x) + 12x^2

In [ ]:
plot(func, -4.0, 4.0)

Si nous commençons assez proche de l'optimum global, nous pouvons le trouver.

In [ ]:
x0 = -3.0
iter = Newton(func, dfunc, d2func, x0, true, true)

In [ ]:
iter

In [ ]:
plot(func, -3.2, -2.0, label="Function")
x = -3.0
m(y) = func(x)+dfunc(x)*(y-x)+0.5*d2func(x)*(y-x)^2
m(-3.0)

In [ ]:
plot!(m, -3.2, -2.0, label="Model")
plot!(iter[1:2,2], iter[1:2,1], label="Newton step")
vline!([iter[1,2] iter[2,2]], label = "")

In [ ]:
plot(func, -3.2, -2.0, label="Function")
plot!(iter[:,2], iter[:,1], label="Newton steps")

In [ ]:
x0 = -4.0
Newton(func, dfunc, d2func, x0, true)

In [ ]:
x0 = -2.2
Newton(func, dfunc, d2func, x0, true)

Cependant, si la méthode converge, nous savons seulement qu'elle converge vers un point où la dérivée s'annule. Il pourrait s'agit d'un maximum local!

In [ ]:
x0 = 1.0
Newton(func, dfunc, d2func, x0, true)

In [ ]:
α = 0.1
x = x0-α*dfunc(x0)/d2func(x0)
α = 0.001
x = x0-α*dfunc(x0)/d2func(x0)

In [ ]:
f(x)

In [ ]:
dfunc(x0)/d2func(x0)

In [ ]:
dfunc(x0)

Nous pouvons aussi converger vers un minimum local, mais pas global.

In [ ]:
x0 = 2.0
Newton(func, dfunc, d2func, x0, true)

## Méthode de la sécante

In [ ]:
function Secant(f::Function, df::Function, x0::Float64, x1::Float64,
                verbose::Bool = false, δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = x0
    y = x1
    if (x0 == x1)
        println("x0 must different from x1")
        return
    end
    if (verbose)
        println("0. x0 = $x0, f($x0) = $(f(x0))")
        println("1. x1 = $x1, f($x1) = $(f(x1))")
    end
    dfx = df(x)
    dfy = df(y)
    while (abs(dfy) > δ && k < nmax)
        k += 1
        t = y
        y = y-(x-y)/(dfx-dfy)*dfy
        x = t
        if (verbose)
            println("$k. x = $y, f(x) = $(f(y))")
        end
        dfx = dfy
        dfy = df(y)        
    end
    
    return y
end

In [ ]:
Secant(func, dfunc, 1.0, 2.0, true)

## Recherche de racine avec la méthode de Newton

Itération:
$$
x_{k+1} = x_k-\frac{f(x)}{f'(x)}
$$

Considérons la fonction

In [ ]:
f(x) = x-2sin(x)

In [ ]:
df(x) = 1-2cos(x)

In [ ]:
plot(f, -5.0, 5.0)

Nous cherchons un zéro de $f(x)$.

In [ ]:
function NewtonRoot(f::Function, df::Function, xstart::Float64,
        verbose::Bool = false, δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = xstart
    fx = f(x)
    if (verbose)
        println("$k. x = $x, f(x) = $fx")
    end
    while (abs(fx) > δ && k < nmax)
        k += 1
        dfx = df(x)
        x = x-fx/df(x)
        fx = f(x)
        if (verbose)
            println("$k. x = $x, f(x) = $fx")
        end
    end
    
    return x
end

In [ ]:
x0 = 1.1
NewtonRoot(f, df, x0, true)

Cela fonctionne, mais nous étions proches d'un désastre! Observons que

In [ ]:
df(1.1)

La courbe est presque plate, et nous sommes entraînés vers un point très éloigné. Par chance, nous revenons sur nos pas, mais en utilisant beaucoup d'itérations.

Considérons à présent le point de départ

In [ ]:
x0 = π/3

La dérivée à ce point est

In [ ]:
df(x0)

La méthode de Newton donne

In [ ]:
NewtonRoot(f, df, x0, true)

Nous sommes moins chanceux! En fait, en $\pi/3$,

In [ ]:
df(π/3)

La récurrence de Newton rencontre des problèmes comme nous avons une division pas zéro. La fonction continue néanmoins comme, en raison des erreurs numériques, nous évitons la division par zéro. Toutefois, la méthode diverge, comme $x \rightarrow -\infty$.

Prenons à présent un point où la dérivée est plus importante.

In [ ]:
x0 = 4.0
NewtonRoot(f, df, x0, true)

La méthode converge à présent très rapidement, même si le point de départ était plus éloigné de la solution.

Notons que $x-2\sin x = 0$ est équivalent à $\frac{1}{\sin x} - \frac{2}{x} = 0$ si nous exigeons que $x \ne k\pi$, $k \in \mathcal{Z}$. La forme de la fonction est cependant différente autour du zéro de la fonction.

In [ ]:
g(x) = 1/sin(x) - 2/x
plot(g, 0.1, π-0.1)

La dérivée est

In [ ]:
dg(x) = 2/(x*x)-cos(x)/(sin(x)^2)

In [ ]:
x0 = 1.1
NewtonRoot(g, dg, x0, true)

Nous observons maintenant une convergence rapide, principalement en raison du fait que la fonction ne présente pas de parties plates.

In [ ]:
f(x) = exp(x/2)-x-1

In [ ]:
df(x) = 0.5*exp(x/2)-1

In [ ]:
x0 = 1.0
NewtonRoot(f, df, x0, true)

Vérifions les résultats obtenus avec la librairie `Roots`.

In [ ]:
using Roots

In [ ]:
y = fzero(df,0.0, 5.0)

In [ ]:
x0 = y
NewtonRoot(f, df, x0, true)

## Cycles

Considérons à présent la fonction

In [ ]:
h(x) = x^3 - 2*x + 2

In [ ]:
plot(h, -2.5, 1.5)

In [ ]:
dh(x) = 3x^2-2

In [ ]:
x0 = 0.0
NewtonRoot(h, dh, x0, true)

In [ ]:
ratio(x) = h(x)/dh(x)

In [ ]:
ratio(0.0)

In [ ]:
ratio(1.0)

La méthode cycle! Cependant, si nous changeons le point de départ, nous pouvons converger.

In [ ]:
x0 = 0.5
NewtonRoot(h, dh, x0, true)

## Application: calcul de la racine carrée d'un nombre non négatif

La racine carrée d'un nombre réel non négatif peut être obtenue à l'aide de la méthode de Newton. Plus précisément, si nous cherchons la racine carrée de $s$, nous pouvons reformuler le problème comme la recherche d'un zéro de la fonction
$$
f(x) = x^2 - s
$$
En effet, si $f(x) = 0$, alors $s = x^2$, ou $x = \pm\sqrt{s}$.

Développons la récurrence de Newton pour ce problème.

Choisir un bon point de départ peut cependant être un problème. (Wikipedia) Avec $s$ exprimé en notation scientifique comme  $a\times 10^{2n}$ avec $1\leq a<100$ et $n$ entier, $\sqrt{s} \approx \sqrt{a} 10^n$. Le point de départ est souvent calculé comme
$$
\sqrt{s} =
\begin{cases}
2\times10^n & \mbox{ if } a < 10\\
6\times10^n & \mbox{ if } a \geq 10
\end{cases}
$$
Les facteurs deux et six sont utilisés car ils permettent d'approximer les moyennes géométriques de la plus petite et la plus grande valeurs pour un nombre donnés de chiffres:
$$
\sqrt{\sqrt{1}\sqrt{10}} = 10^{\frac{1}{4}} \approx 2
$$
et
$$
\sqrt{\sqrt{10}\sqrt{100}} = 10^{\frac{3}{4}} \approx 6
$$

In [ ]:
sq(x,s) = x*x-s

In [ ]:
dsq(x) = 2*x

In [ ]:
sqs(x) = x*x-s

In [ ]:
s = 6.0

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, true)

Nous pouvons vérifier la solution comme

In [ ]:
x^2

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-8)

In [ ]:
x^2

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-10)
x^2

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-12)
x^2

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-14)
x^2

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-16)
x^2

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-15)
x^2

In [ ]:
6-x^2

In [ ]:
eps()

In [ ]:
s = 25
x = NewtonRoot(sqs, dsq, 6.0, true, 1e-15)
x^2

In [ ]:
s = 400
x = NewtonRoot(sqs, dsq, 6.0, true, 1e-15)
x^2